**pixyz  
last update 2022 03 16  
ゆっくりしていってね！**<br>
**ver4 Changed input data to stage2**

# Config

In [ ]:
#stage1 BASE_PATH = "../input/womens-march-mania-2022/WDataFiles_Stage1/"
BASE_PATH = "../input/womens-march-mania-2022/WDataFiles_Stage2/"

TRESULT_NAME = "WNCAATourneyDetailedResults.csv"
SRESULT_NAME = "WRegularSeasonDetailedResults.csv"
SEED_NAME = "WNCAATourneySeeds.csv"
TEST_NAME = "WSampleSubmissionStage2.csv"

In [ ]:
class config:    
    
    SEED = 42
    FOLD_TO_RUN = 0
    FOLDS = 5
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    
    model_type = 'lgb_regression'  
    message='baseline'
    TARGET = 'target'
    
    TEST = False

In [ ]:
if config.TEST:
    config.EPOCHS = 100

In [ ]:
lgb_params = {
    'objective': 'regression',
    'n_estimators': 20000,
    'random_state': 42,
    'learning_rate': 2e-4,
    'subsample': 0.44,
    'subsample_freq': 1,
    'colsample_bytree': 0.64,
    'reg_alpha': 0.07,
    'reg_lambda': 0.07,
    'max_depth':100,
    'num_leaves':356,
    'min_child_weight': 256,
    'min_child_samples': 72,
    'device':'gpu'
}

**魔理沙:今回はFOLD5、EPOCH20000で学習をしていくぜ**  

**Marisa: This time, I'll study with FOLD5 and EPOCH20000**

# Import

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import matplotlib.pylab as plt
import seaborn as sns

**霊夢：今回は回帰タスクのコンペかな？**

**魔理沙：まずは概要を読んでみよう。**

<br>

**Reimu: Is this a regression task competition?**

**Marisa: First, let's read the overview.**

別の年、動揺を予測し、確率を呼び出し、ブラケット学のスキルをリーダーボードテストにかける別のチャンス。3月に開催される第8回機械学習マニア大会では、カグラーズが今年の米国女子大学バスケットボールトーナメントの結果を予測しようとする何百万人ものファンに再び参加します。しかし、ほとんどのファンとは異なり、豊富な履歴データと計算能力の組み合わせを使用して勝者と敗者を選び、テレビでグラウンドトゥルースが展開されます。

過去のNCAAゲームのデータが提供されており、他の公開されているデータソースを使用して勝利を収めることをお勧めします。

この2段階のコンテストの第1段階では、参加者は以前のトーナメントに対してモデルを作成してテストします。第2段階では、参加者は2022年のトーナメントの結果を予測します。2番目のステージに入るのに最初のステージに参加する必要はありません。最初の段階は、モデル構築を奨励し、予測をスコアリングする手段を提供するために存在します。実際の競争は2022年の結果を予測しています。

そして、米国の男子大学バスケットボールトーナメントの結果を予測するために見られる私たちのコンパニオンコンペティションを見るのを忘れないでください！

Another year, another chance to predict the upsets, call the probabilities, and put your bracketology skills to the leaderboard test. In our eighth annual March Machine Learning Mania competition, Kagglers will once again join the millions of fans who attempt to predict the outcomes of this year's US women's college basketball tournament. But unlike most fans, you will pick the winners and losers using a combination of rich historical data and computing power, while the ground truth unfolds on television.

You're provided data of historical NCAA games and are encouraged to use other sources of publicly available data to gain a winning edge.

In stage one of this two-stage competition, participants will build and test their models against previous tournaments. In the second stage, participants will predict the outcome of the 2022 tournament. You don’t need to participate in the first stage to enter the second. The first stage exists to incentivize model building and provide a means to score predictions. The real competition is forecasting the 2022 results.

And don't forget to take a look at our companion competition that looks to predict the outcome of the US men's college basketball tournament!

<img src = "https://storage.googleapis.com/kaggle-competitions/kaggle/4862/media/bball-logo.png" width = 700>

**魔理沙:今回はlightgbmを使ったbaselineを作っていくよ。**

**霊夢：今回はデータの種類がとても多いね。あれ？trainデータはどこ？**

**魔理沙:trainデータは自分でいくつかのデータを組み合わせて作るしかないみたいだな。これは骨が折れそうだ。**

<br>

**Marisa: This time I'm going to make a baseline using lightgbm.**

**Reimu: There are so many types of data this time. that? Where is the train data?**

**Marisa: It seems that train data can only be created by combining some data. This is going to be a pain.**

<img src = "https://4.bp.blogspot.com/-uoVuBWIbdiA/WvQHqpx_YCI/AAAAAAABL8g/NiFZ6K71VBc_0_dcKb3_4nhnvFJ_JMNuACLcBGAs/s450/network_dennou_sekai_figure.png" width = 200>

# Data

**魔理沙:データは以下の通りだぜ。**  

**Marisa: The data is as follows.**

In [ ]:
!ls ../input/womens-march-mania-2022/WDataFiles_Stage1

**霊夢:一番大事なデータはどれだろう？**

**魔理沙:まずはMNCAATourneyDetailedResultsかな。これを主軸にやっていくぜ。**

<br>

**Reimu: Which is the most important data?**

**Marisa: First of all, MNCAA Tourney Detailed Results. Let's do this as the main axis.**

In [ ]:
df_TDresults = pd.read_csv(BASE_PATH + TRESULT_NAME)
df_RSDresults = pd.read_csv(BASE_PATH + SRESULT_NAME)
df_TDresults.head()

**霊夢:このままだとデータ量が少ないね。**

**魔理沙:そうだね、だからMRegularSeasonDetailedResultsも追加して量を増やそう。**

**霊夢:これでうまくいくといいな―**

<br>

**Reimu: The amount of data is small at this rate.**

**Marisa: That's right, so let's add MRegularSeasonDetailedResults to increase the amount.**

**Reimu: Hope this works**

<img src="https://1.bp.blogspot.com/-5i9P5bsm_CA/Xbo6PsHbZ-I/AAAAAAABVvg/5uwfw-rUEt4VeLDEbS4YUIViH3SkealiACNcBGAsYHQ/s400/yaruki_aru_casual_woman.png" width = 200>

In [ ]:
df_TDresults = pd.merge(df_RSDresults,df_TDresults,on = df_TDresults.columns.tolist(),how = "outer")
len(df_TDresults)

**霊夢:FGMとかFGAって何のこと？**

**魔理沙:バスケには詳しくないから具体的には分からないが、どうやらどれもゲーム内で起こした特定のアクションの数みたいだね。**

**霊夢:試合ごとにばらばらだからどうやって扱おうかな？**

**魔理沙:とりあえずSeason、TeamIDごとに平均をとって特徴量にしよう。上手くいくかはちょっとわからないかな。**

<br>

**Reimu: What is FGM or FGA?**

**Marisa: I'm not familiar with basketball, so I don't know the specifics, but it seems that all of them seem to be the number of specific actions taken in the game.**  
**Reimu: How do you handle it because it's different for each match?**

**Marisa: For the time being, let's take the average for each Season and Team ID and use it as a feature. I don't know if it will work.**

<img src="http://3.bp.blogspot.com/-KmQQLtEkmLw/U1T3r7D0NdI/AAAAAAAAfVI/c2d4n2kG00U/s400/figure_question.png" width="200">

In [ ]:
box_col = ["FGM","FGA","FGM3","FTM","FTA","OR","DR","Ast","TO","Stl","Blk","PF"]
df_boxW = df_TDresults[["Season","WTeamID"]+["W" + col for col in box_col]]
df_boxL = df_TDresults[["Season","LTeamID"]+["L" + col for col in box_col]]
df_boxW = df_boxW.rename(columns={"WTeamID":"TeamID"})
df_boxW = df_boxW.rename(columns={("W"+ col):col for col in box_col})
df_boxL = df_boxL.rename(columns={"LTeamID":"TeamID"})
df_boxL = df_boxL.rename(columns={("L"+ col):col for col in box_col})
df_box = pd.merge(df_boxW,df_boxL,on = ["Season","TeamID"]+box_col,how="outer")
df_box = df_box.groupby(["Season","TeamID"])[box_col].agg(np.mean).reset_index()
df_box.head()

**霊夢:そういえばこのままだと絶対左側のチームIDが勝つことになっちゃうじゃん！**<br>

**魔理沙:そうだった。TeamIDなどのデータを勝ったチームと負けたチームを反転させたデータを下にくっつけよう。これでデータ量も2倍になるぞ。**

<br>

**Reimu: By the way, the team ID on the left side will definitely win!** <br>

**Marisa: That's right. Let's attach the data such as Team ID, which is the reverse of the winning team and the losing team, to the bottom. This will double the amount of data.**

<img src="https://4.bp.blogspot.com/-WOvX4GkiSEo/U1T3wVr3ovI/AAAAAAAAfWg/RpbQYAHDo2Y/s400/figure_shock.png" width = 200>

In [ ]:
df_TDresults2 = df_TDresults
df_TDresults = df_TDresults.rename(columns={"WTeamID":"Team1ID","LTeamID":"Team2ID","WScore":"T1Score","LScore":"T2Score"})
df_TDresults = df_TDresults.rename(columns={f"W{col}":f"T1{col}" for col in box_col})
df_TDresults = df_TDresults.rename(columns={f"L{col}":f"T2{col}" for col in box_col})
df_TDresults2 = df_TDresults2.rename(columns={"WTeamID":"Team2ID","LTeamID":"Team1ID","WScore":"T2Score","LScore":"T1Score"})

**霊夢:targetはどうするの？**

**魔理沙:Team1が勝ったときは1.0、負けたときは0.0にしよう。**

<br>

**Reimu: What about target?**

**Marisa: Let's set it to 1.0 when Team 1 wins and 0.0 when it loses.**

In [ ]:
features = ["Season","Team1ID","Team2ID","T1Score","T2Score",'target']
df_TDresults['target'] = 1.0
df_TDresults2['target'] = 0.0
train = pd.merge(df_TDresults,df_TDresults2,on = features,how="outer")
train = train[features]
train.head()

In [ ]:
box_T1 = df_box.copy()
box_T2 = df_box.copy()
box_T1.columns = ['Season','Team1ID'] + ["T1"+col+"_mean" for col in box_col]
box_T2.columns = ['Season','Team2ID'] + ["T2"+col+"_mean" for col in box_col]
train = pd.merge(train,box_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,box_T2,on = ["Season","Team2ID"],how = "left")

**霊夢:今度は何をするの？**

**魔理沙:次はseedを特徴量として組み込もう！**

<br>

**Reimu: What are you going to do next time?**

**Marisa: Next, let's incorporate seed as a feature!**

<img src = "https://1.bp.blogspot.com/-FrFmmMlJ0bE/U1T3wsYJlRI/AAAAAAAAfWo/7-7pGdnlkKQ/s400/figure_talking.png" width = 200>

In [ ]:
df_seeds = pd.read_csv(BASE_PATH + SEED_NAME)
df_seeds.head()

**霊夢:それぞれのチームのseedが独立した特徴量で正しく学習できるのかな？**

**魔理沙:ちょっと心配だね。seedの差を取った特徴量も作っておこう**

<br>

**Reimu: Can the seeds of each team learn correctly with independent features?**

**Marisa: I'm a little worried. Let's make a feature quantity with the difference in seed**

In [ ]:
df_seeds["seed"] = df_seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T2 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','Team1ID','T1_seed']
seeds_T2.columns = ['Season','Team2ID','T2_seed']
train = pd.merge(train,seeds_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,seeds_T2,on = ["Season","Team2ID"],how = "left")
train["seeddiff"] = train["T1_seed"] - train["T2_seed"]

In [ ]:
features = train.drop(columns = ["target","T1Score","T2Score"]).columns
train.head()

**霊夢:これで一通りのデータができあがったね。**

**魔理沙：でもまだ終わりじゃないよ、testデータも同様にして作っていこう。**

**霊夢：ああ大変だあ**

<br>

**Reimu: That's all the data.**

**Marisa: But it's not over yet, let's make test data in the same way.**

**Reimu: Oh, it's hard**

<img src="https://3.bp.blogspot.com/-3q31Kj8zFHc/V6BP6vEGe1I/AAAAAAAA83s/YD0NZlG5wmgMiB_iX9QMDy8KF2IEid6qQCLcB/s450/figure_korobu.png" width="200">

In [ ]:
test = pd.read_csv(BASE_PATH + TEST_NAME)
test.head()

In [ ]:
test["Season"] = test['ID'].apply(lambda x: int(x[0:4]))
test["Team1ID"] = test['ID'].apply(lambda x: int(x[5:9]))
test["Team2ID"] = test['ID'].apply(lambda x: int(x[10:14]))
test.head()

In [ ]:
test = pd.merge(test,box_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,box_T2,on = ["Season","Team2ID"],how = "left")
test.head()

In [ ]:
test = pd.merge(test,seeds_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,seeds_T2,on = ["Season","Team2ID"],how = "left")
test["seeddiff"] = test["T1_seed"] - test["T2_seed"]

In [ ]:
test = test.drop(columns = ['ID','Pred'])
test.head()

# Model&Train

**霊夢:やっと学習だ！**<br>

**魔理沙:上手くいくかなあ**

<br>

**Reimu: It's finally learning!**<br>

**Marisa: I wonder if it will work**

<img src="https://1.bp.blogspot.com/-flCQBhvvg6U/V6BP7KK5RdI/AAAAAAAA830/exz5qqb8M0U-hyss5tb4p6St5nF9aq0-wCLcB/s800/figure_ouen.png" width = 200>

In [ ]:
lgb_oof = np.zeros(train.shape[0])
val_pred = np.zeros(train.shape[0])
lgb_pred = np.zeros(test.shape[0])
lgb_importances = pd.DataFrame()
models = []
skf = StratifiedKFold(n_splits=config.FOLDS, shuffle = True , random_state=config.SEED)

for fold, (trn_idx, val_idx) in enumerate(skf.split(X=train, y=train['Season'])):
    print(f"===== fold {fold} =====")
    X_train,y_train = train[features].iloc[trn_idx],train[config.TARGET].iloc[trn_idx]
    X_valid,y_valid = train[features].iloc[val_idx],train[config.TARGET].iloc[val_idx]
    start = time.time()
    model = LGBMRegressor(**lgb_params)
    model.fit(
        X_train,
        y_train,
        eval_set = (X_valid,y_valid),
        eval_metric = 'rmse',
        early_stopping_rounds=config.EARLY_STOPPING_ROUNDS,
        verbose=config.VERBOSE,
    )
    fi_tmp = pd.DataFrame()
    fi_tmp['feature'] = model.feature_name_
    fi_tmp['importance'] = model.feature_importances_
    fi_tmp['fold'] = fold
    fi_tmp['seed'] = config.SEED
    lgb_importances = lgb_importances.append(fi_tmp)
    val_pred = model.predict(X_valid) 
    lgb_pred += model.predict(test)/config.FOLDS
    elapsed = time.time() - start
    rmse = np.sqrt(mean_squared_error(y_valid, val_pred))
    print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")
    models.append(model)

# print(f"oof lgb roc = {accuracy_score(train[config.TARGET], lgb_oof)}")

# np.save("lgb_oof.npy", lgb_oof)
# np.save("lgb_pred.npy", lgb_pred)

# Evaluation

**霊夢:どの特徴量をよく使ったか見てみよう！**

**魔理沙:頑張って作った特徴量が活きてるといいなあ。**

<br>

**Reimu: Let's see which features you used often!**<br>

**Marisa: I hope the features I made with all my might are alive.**

In [ ]:
order = list(lgb_importances.groupby('feature').mean().sort_values('importance', ascending=False).index)

fig = plt.figure(figsize=(16, 32), tight_layout=True)
sns.barplot(x="importance", y="feature", data=lgb_importances.groupby('feature').mean().reset_index(), order=order)
plt.title("LightGBM feature importances")

**霊夢:rankが結構使われているみたいだね。**

**魔理沙:そうだね逆にseedはあんまりかな。**

<br>

**Reimu: It seems that rank is used quite a bit.**

**Marisa: That's right, on the contrary, seed is not so much.**

<img src="https://1.bp.blogspot.com/-N08ojzWTStM/VdLrmt8nTzI/AAAAAAAAwuU/1o3JOcdtHNM/s400/business_senryaku_sakuryaku_man.png" width=200>

# Inference

**魔理沙:最後にsubmissionファイルを作ろう！**<br>

**霊夢:最後まで頑張るぞ！**

<br>

**Marisa: Finally, let's create a submission file!** <br>

**Reimu: I'll do my best until the end!**

<img src = "https://4.bp.blogspot.com/-mg-ul7z64Oc/WzC-BvpjsgI/AAAAAAABM94/-OPzexySWfUXUdf6qzZ45CfERCUyoNpBQCLcBGAs/s400/yaruki_moeru_woman.png" width=200>

In [ ]:
for pred_num in range(len(lgb_pred)):
    if lgb_pred[pred_num]<0:
        lgb_pred[pred_num]=0
    if lgb_pred[pred_num]>1:
        lgb_pred[pred_num]=1
        
df_submission = pd.read_csv(BASE_PATH + TEST_NAME)
df_submission['Pred'] = lgb_pred
df_submission.to_csv("submission.csv",index = None)
df_submission.head()

**霊夢:どんな特徴量を作ったら精度が上がるのかな？**

**魔理沙:それはたくさんの人のcodeを見て学ばないとね。**

**霊夢:もっと頑張らなくちゃだね。**

<br>

**Reimu: What kind of features should I make to improve the accuracy?**

**Marisa: It has to be learned by looking at the codes of many people.**

**Reimu: I have to work harder.**

****
**霊夢&魔理沙:ご覧いただきありがとうございました！良ければUpvoteをよろしくお願いします！**<br>

**Reimu & Marisa: Thank you for visiting! If you like, thank you for Upvote!**

![](https://1.bp.blogspot.com/-PNcKwFw1PpM/U1T3oDIr9CI/AAAAAAAAfT4/gEn86X8Ppx0/s400/figure_goodjob.png)